In [1]:
import warnings
warnings.filterwarnings("ignore")
from copy import deepcopy
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import sys

sys.path.append("/code")

from tqdm import tqdm
import torch
# device = torch.device('cpu')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# import gym
# import recogym

import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

torch.backends.cudnn.benchmark = torch.cuda.is_available()
if torch.cuda.is_available():
    torch.set_float32_matmul_precision("high")  # TF32 = big speedup on Ada


from sklearn.utils import check_random_state

# implementing OPE of the IPWLearner using synthetic bandit data
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt

from scipy.special import softmax
import optuna
# from memory_profiler import profile


from estimators import (
    DirectMethod as DM
)

from simulation_utils import (
    eval_policy,
    generate_dataset,
    create_simulation_data_from_pi,
    get_train_data,
    get_opl_results_dict,
    CustomCFDataset,
    calc_reward
)

from models import (    
    CFModel,
    NeighborhoodModel,
    BPRModel, 
    RegressionModel
)

from training_utils import (
    fit_bpr,
    train,
    validation_loop
 )

from custom_losses import (
    SNDRPolicyLoss,
    BPRLoss
    )

random_state=12345
random_ = check_random_state(random_state)

pd.options.display.float_format = '{:,.8f}'.format

Using device: cuda
Using device: cuda
Using device: cuda


## `trainer_trial` Function

This function runs policy learning experiments using offline bandit data and evaluates various estimators.

### Parameters
- **num_runs** (int): Number of experimental runs per training size
- **num_neighbors** (int): Number of neighbors to consider in the neighborhood model
- **num_rounds_list** (list): List of training set sizes to evaluate
- **dataset** (dict): Contains dataset information including embeddings, action probabilities, and reward probabilities
- **batch_size** (int): Batch size for training the policy model
- **num_epochs** (int): Number of training epochs for each experiment
- **lr** (float, default=0.001): Learning rate for the optimizer

### Process Flow
1. Initializes result structures and retrieval models
2. For each training size in `num_rounds_list`:
   - Creates a uniform logging policy and simulates data
   - Generates training data for offline learning
   - Fits regression and neighborhood models for reward estimation
   - Initializes and trains a counterfactual policy model
   - Evaluates policy performance using various estimators
   - Collects metrics on policy reward and embedding quality

### Returns
- **DataFrame**: Results table with rows indexed by training size and columns for various metrics:
  - `policy_rewards`: True expected reward of the learned policy
  - Various estimator errors (`ipw`, `reg_dm`, `conv_dm`, `conv_dr`, `conv_sndr`)
  - Variance metrics for each estimator
  - Embedding quality metrics comparing learned representations to ground truth

### Implementation Notes
- Uses uniform random logging policy for collecting offline data
- Employs Self-Normalized Doubly Robust (SNDR) policy learning
- Measures embedding quality via RMSE to original/ground truth embeddings

In [ ]:
def trainer_trial(
    num_runs,
    num_neighbors,
    num_rounds_list,
    dataset,
    batch_size,
    val_size=2000
):
    import torch
    from torch.utils.data import DataLoader

    # ---- Device & CUDA fast-paths ----
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    torch.backends.cudnn.benchmark = torch.cuda.is_available()
    if torch.cuda.is_available():
        torch.set_float32_matmul_precision("high")  # TF32 on Ada

    dm = DM()
    results = {}

    our_x, our_a = dataset["our_x"], dataset["our_a"]
    emb_x, emb_a = dataset["emb_x"], dataset["emb_a"]
    original_x, original_a = dataset["original_x"], dataset["original_a"]
    n_users, n_actions, emb_dim = dataset["n_users"], dataset["n_actions"], dataset["emb_dim"]

    ### NEW: indices once
    all_user_indices = np.arange(n_users, dtype=np.int64)

    def T(x):
        return torch.as_tensor(x, device=device, dtype=torch.float32)

    # ---- Optuna objective uses train/val loaders from outer scope ----
    def objective(trial):
        lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
        epochs = trial.suggest_int("num_epochs", 1, 10)

        trial_neigh_model = NeighborhoodModel(
            train_data['x_idx'], train_data['a'],
            our_a, our_x, train_data['r'],
            num_neighbors=num_neighbors
        )

        ### NEW: build trial_scores_all ONCE and keep on device
        trial_scores_all = torch.as_tensor(
            trial_neigh_model.predict(all_user_indices),   # shape [n_users, n_actions]
            device=device, dtype=torch.float32
        )

        trial_model = CFModel(
            n_users, n_actions, emb_dim,
            initial_user_embeddings=T(our_x),
            initial_actions_embeddings=T(our_a)
        ).to(device)

        assert (not torch.cuda.is_available()) or next(trial_model.parameters()).is_cuda

        ### CHANGED: pass trial_scores_all
        train(
            trial_model, train_loader, trial_neigh_model, trial_scores_all,
            criterion=SNDRPolicyLoss(), num_epochs=epochs, lr=lr, device=device
        )
        val =  validation_loop(trial_model, val_loader, trial_neigh_model, trial_scores_all, device=device)
        if not torch.isfinite(torch.tensor(val)):
            raise optuna.TrialPruned()  # or return a sentinel very-low score
        return val

    first = True

    for train_size in num_rounds_list:
        reg_results, conv_results = [], []

        for run in range(num_runs):
            pi_0 = softmax(our_x @ our_a.T, axis=1)
            original_policy_prob = np.expand_dims(pi_0, -1)

            simulation_data = create_simulation_data_from_pi(
                dataset, pi_0, train_size + val_size,
                random_state=(run + 1) * train_size
            )

            idx = np.arange(train_size)

            train_data = get_train_data(n_actions, train_size, simulation_data, idx, our_x)
            val_data   = get_train_data(n_actions, val_size, simulation_data, np.arange(val_size) + train_size, our_x)

            regression_model = RegressionModel(
                n_actions=n_actions, action_context=our_x,
                base_model=LogisticRegression(random_state=12345)
            )
            regression_model.fit(
                train_data['x'], train_data['a'], train_data['r'],
                original_policy_prob[train_data['x_idx'], train_data['a']].squeeze()
            )

            neighberhoodmodel = NeighborhoodModel(
                train_data['x_idx'], train_data['a'],
                our_a, our_x, train_data['r'],
                num_neighbors=num_neighbors
            )

            ### NEW: build scores_all ONCE per NeighborhoodModel and keep it on device
            scores_all = torch.as_tensor(
                neighberhoodmodel.predict(all_user_indices),   # [n_users, n_actions]
                device=device, dtype=torch.float32
            )

            model = CFModel(
                n_users, n_actions, emb_dim,
                initial_user_embeddings=T(our_x),
                initial_actions_embeddings=T(our_a)
            ).to(device)

            # DataLoaders: feed the GPU
            num_workers = 4 if torch.cuda.is_available() else 0
            cf_dataset = CustomCFDataset(
                train_data['x_idx'], train_data['a'], train_data['r'], original_policy_prob
            )
            train_loader = DataLoader(
                cf_dataset, batch_size=batch_size, shuffle=True,
                pin_memory=torch.cuda.is_available(),
                num_workers=num_workers, persistent_workers=bool(num_workers)
            )

            val_dataset = CustomCFDataset(
                val_data['x_idx'], val_data['a'], val_data['r'], original_policy_prob
            )
            val_loader = DataLoader(
                val_dataset, batch_size=val_size, shuffle=False,
                pin_memory=torch.cuda.is_available(),
                num_workers=num_workers, persistent_workers=bool(num_workers)
            )

            if first:
                policy = np.expand_dims(softmax(our_x @ our_a.T, axis=1), -1)
                conv_results.append(eval_policy(neighberhoodmodel, train_data, original_policy_prob, policy))
                conv_results[-1] = np.append(calc_reward(dataset, policy), conv_results[-1])
                conv_results[-1] = np.append(conv_results[-1], [np.sqrt(np.mean((emb_a-our_a)**2)), np.sqrt(np.mean((original_a-our_a)**2))])
                conv_results[-1] = np.append(conv_results[-1], [np.sqrt(np.mean((emb_x-our_x)**2)), np.sqrt(np.mean((original_x-our_x)**2))])
                reg_dm = dm.estimate_policy_value(policy[train_data['x_idx']], regression_model.predict(train_data['x']))
                reg_results.append(reg_dm)
                first = False
                reg_results = np.array(reg_results)
                conv_results = np.array(conv_results)
                results[0] = get_opl_results_dict(reg_results, conv_results)
                reg_results, conv_results = [], []

            # ---- Hyperparam search ----
            study = optuna.create_study(direction="maximize")
            study.optimize(objective, n_trials=10, show_progress_bar=True)
            best_params = study.best_params

            # ---- Retrain with best params ----
            neighberhoodmodel = NeighborhoodModel(
                train_data['x_idx'], train_data['a'],
                our_a, our_x, train_data['r'],
                num_neighbors=num_neighbors
            )
            scores_all = torch.as_tensor(
                neighberhoodmodel.predict(all_user_indices),
                device=device, dtype=torch.float32
            )

            model = CFModel(
                n_users, n_actions, emb_dim,
                initial_user_embeddings=T(our_x),
                initial_actions_embeddings=T(our_a)
            ).to(device)
            assert (not torch.cuda.is_available()) or next(model.parameters()).is_cuda

            ### CHANGED: pass scores_all
            train(
                model, train_loader, neighberhoodmodel, scores_all,
                criterion=SNDRPolicyLoss(),
                num_epochs=best_params['num_epochs'], lr=best_params['lr'],
                device=device
            )

            # Pull updated params back to CPU numpy
            our_x_t, our_a_t = model.get_params()
            our_a, our_x = our_a_t.detach().cpu().numpy(), our_x_t.detach().cpu().numpy()

            policy = np.expand_dims(softmax(our_x @ our_a.T, axis=1), -1)
            reg_dm = dm.estimate_policy_value(policy[train_data['x_idx']], regression_model.predict(train_data['x']))
            reg_results.append(reg_dm)

            conv_results.append(eval_policy(neighberhoodmodel, train_data, original_policy_prob, policy))
            conv_results[-1] = np.append(calc_reward(dataset, policy), conv_results[-1])
            conv_results[-1] = np.append(conv_results[-1], [np.sqrt(np.mean((emb_a-our_a)**2)), np.sqrt(np.mean((original_a-our_a)**2))])
            conv_results[-1] = np.append(conv_results[-1], [np.sqrt(np.mean((emb_x-our_x)**2)), np.sqrt(np.mean((original_x-our_x)**2))])

            # reset the working embeddings for next run
            our_a, our_x = original_a.copy(), original_x.copy()

        torch.cuda.empty_cache()
        reg_results = np.array(reg_results)
        conv_results = np.array(conv_results)
        results[train_size] = get_opl_results_dict(reg_results, conv_results)

    return pd.DataFrame.from_dict(results, orient='index')


## Learning

We will run several simulations on a generated dataset, the dataset is generated like this:
$$ \text{We have users U and actions A } u_i \sim N(0, I_{emb_dim}) \ a_i \sim N(0, I_{emb_dim})$$
$$ p_{ij} = 1 / (5 + e^{-(u_i.T a_j)}) $$
$$r_{ij} \sim Bin(p_{ij})$$

We have a policy $\pi$
and it's ground truth reward is calculated by
$$R_{gt} = \sum_{i}{\sum_{j}{\pi_{ij} * p_{ij}}} $$

Our parameters for the dataset will be
$$EmbDim = 5$$
$$NumActions= 150$$
$$NumUsers = 150$$
$$NeighborhoodSize = 6$$

to learn a new policy from $\pi$ we will sample from:
$$\pi_{start} = (1-\epsilon)*\pi + \epsilon * \pi_{random}$$

In [3]:
dataset_params = dict(
                    n_actions= 500,
                    n_users = 500,
                    emb_dim = 16,
                    # sigma = 0.1,
                    eps = 0.6, # this is the epsilon for the noise in the ground truth policy representation
                    ctr = 0.2
                    )

train_dataset = generate_dataset(dataset_params)

Random Item CTR: 0.12972795060603162
Optimal greedy CTR: 0.19999707792821972
Optimal Stochastic CTR: 0.19982996880994605
Our Initial CTR: 0.1646085673501415


In [4]:
train_dataset.keys()

dict_keys(['emb_a', 'our_a', 'original_a', 'emb_x', 'our_x', 'original_x', 'q_x_a', 'n_actions', 'n_users', 'emb_dim', 'user_prior'])

In [5]:
num_runs = 1
batch_size = 200
num_neighbors = 6
num_rounds_list = [100, 1000, 5000]

### 1

$$emb = 0.7 * gt + 0.3 * noise$$
$$lr = 0.005$$
$$n_{epochs} = 1$$
$$BatchSize=50$$

In [ ]:
print("Type of num_rounds_list:", type(num_rounds_list))
print("Value of num_rounds_list:", num_rounds_list)
df4 = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=35000)

Type of num_rounds_list: <class 'list'>
Value of num_rounds_list: [100, 1000, 5000]


[I 2025-08-20 10:58:13,524] A new study created in memory with name: no-name-8815933b-afdf-41e7-b27a-13d00d3f623a
  0%|          | 0/10 [00:01<?, ?it/s]

[epoch ? step 1/1] alloc=19MB reserved=24MB
[epoch 1/9] alloc=18MB peak=19MB


  0%|          | 0/10 [00:01<?, ?it/s]

[epoch ? step 1/1] alloc=19MB reserved=24MB
[epoch 2/9] alloc=18MB peak=20MB


...

[epoch ? step 1/25] alloc=20MB reserved=9714MB
[epoch ? step 5/25] alloc=20MB reserved=9714MB
[epoch ? step 10/25] alloc=20MB reserved=9714MB
[epoch ? step 15/25] alloc=20MB reserved=9714MB
[epoch ? step 20/25] alloc=20MB reserved=9714MB


Best trial: 4. Best value: 0.128348:  80%|████████  | 8/10 [03:11<00:42, 21.32s/it]

[epoch ? step 25/25] alloc=20MB reserved=9714MB
[epoch 2/2] alloc=19MB peak=23MB


Best trial: 4. Best value: 0.128348:  90%|█████████ | 9/10 [03:12<00:21, 21.09s/it]

[I 2025-08-20 11:02:54,319] Trial 8 finished with value: 0.1061451463843841 and parameters: {'lr': 0.006132607522126743, 'num_epochs': 2}. Best is trial 4 with value: 0.12834802027837242.


In [ ]:
df4[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

In [ ]:
df4

In [ ]:
num_rounds_list = [3000, 6000, 8000, 9000]

### 2

$$emb = 0.7 * gt + 0.3 * noise$$
$$lr = 0.001$$
$$n_{epochs} = 1$$
$$BatchSize=50$$

In [ ]:
df5 = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=3500)

In [ ]:
df5

### 3

$$emb = 0.7 * gt + 0.3 * noise$$
$$lr = 0.003$$
$$n_{epochs} = 10$$
$$BatchSize=50$$

In [ ]:
df6 = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size)

In [ ]:
df6

### 4

$$emb = 0.7 * gt + 0.3 * noise$$
$$lr = 0.05$$
$$n_{epochs} = 10$$
$$BatchSize=150$$

In [ ]:
df7 = trainer_trial(num_runs, num_neighbors, num_rounds_list[:-3], train_dataset, batch_size+100, num_epochs=10, lr=0.05)

In [ ]:
df7